This notebook implements method 2 for matrix factorization (using stochastic gradient descent with bias terms for each user and movie) and enables visualization of the resulting latent factors.

In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
font_kwargs = {'family': 'sans-serif',
               'sans-serif': 'Arial',
               'size': 12}
plt.rc('font', **font_kwargs)
mathtext_kwargs = {'fontset': 'custom',
                   'bf': 'Arial:bold',
                   'cal': 'Arial:italic',
                   'it': 'Arial:italic',
                   'rm': 'Arial'}
plt.rc('mathtext', **mathtext_kwargs)
savefig_kwargs = {'dpi': 300, 'bbox_inches': 'tight',
                  'transparent': True}
plt.rc('pdf', fonttype=42)
#%config InlineBackend.figure_format = 'pdf'
%config InlineBackend.print_figure_kwargs = savefig_kwargs

First, we define functions to train a model with <i>k</i> latent factors, identifying <i>k</i>&times;<i>m</i> matrix U and <i>k</i>&times;<i>n</i> matrix V representing the <i>m</i> users and <i>n</i> movies.

In [38]:
def grad(Y_ij, U_i, V_j, a_i, b_j, mu=0, reg=0):
    '''
    Takes as input training point Y_ij (rating of ith user for
    jth movie), user vector U_i (ith column of U), movie vector
    V_j (jth column of V), user deviation a_i, and movie deviation
    b_j, with optional arguments for global bias mu (assumed to be
    0 by default) and regularization strength reg (set to 0 by
    default).
    
    Returns the gradients of the regularized squared loss function
    with respect to U_i, V_j, a_i, and b_j.
    '''
    grad_U = reg * U_i - (Y_ij - mu - np.dot(U_i, V_j) - a_i - b_j) * V_j
    grad_V = reg * V_j - (Y_ij - mu - np.dot(U_i, V_j) - a_i - b_j) * U_i
    grad_a = reg * a_i - (Y_ij - mu - np.dot(U_i, V_j) - a_i - b_j)
    grad_b = reg * b_j - (Y_ij - mu - np.dot(U_i, V_j) - a_i - b_j)
    
    return grad_U, grad_V, grad_a, grad_b

def err(Y, U, V, a, b, mu=None, reg=0):
    '''
    Takes as input Y, a matrix where each row (i, j, Y_ij) gives the
    rating Y_ij of user i for movie j; U, a matrix of shape (k, m)
    where each column represents one of the m users; V, a matrix of
    shape (k, n) where each column represents one of the n movies;
    a, a vector of shape (m, ) where each value represents the
    user-specific deviation from the global mean; b, a vector of shape
    (n, ) where each value represents the movie-specific deviation from
    the global mean; mu, a scalar giving the global bias for ratings
    (computed from Y if set to None); and reg, a scalar giving the
    regularization strength (set to 0 by default).
    
    Returns the mean regularized squared error for predictions made
    by estimating Y_ij as the dot product of the ith column of U and
    the jth column of V (accounting for global, user, and movie biases).
    '''
    # Compute term for regularization
    err_r = reg * (np.linalg.norm(U) + np.linalg.norm(V) +
                   np.linalg.norm(a) + np.linalg.norm(b)) / 2
    
    # Compute term for squared loss
    if mu is None:
        mu = np.mean(Y_ij[:, -1])
    resid = np.array([Y_ij - mu - np.dot(U[:, i], V[:, j]) - a[i] - b[j]
                      for (i, j, Y_ij) in Y])
    err_s = np.sum(resid ** 2) / 2
    
    # Return mean regularized squared error
    return (err_r + err_s) / len(Y)

def process_Y(Y_in):
    '''
    Takes as input Y_in, a matrix where each row (i, j, Y_ij) gives the
    rating Y_ij of user i for movie j.
    
    Returns Y, a matrix of the same size as Y_in where user and movie IDs
    have been mapped to sorted indices (lowest value has index 0, etc.).
    '''
    Y = np.zeros(Y_in.shape)
    
    # Map user and movie IDs to indices
    ids_U = np.unique(Y_in[:, 0]).astype(int)
    inds_U = -np.ones(np.max(ids_U) + 1)
    for i, id_U in enumerate(ids_U):
        inds_U[id_U] = i
    ids_V = np.unique(Y_in[:, 1]).astype(int)
    inds_V = -np.ones(np.max(ids_V).astype(int) + 1)
    for i, id_V in enumerate(ids_V):
        inds_V[id_V] = i
    
    # Convert IDs in Y_in to indices
    for r, (i, j, Y_ij) in enumerate(Y_in):
        Y[r] = [inds_U[i], inds_V[j], Y_ij]
    Y = Y.astype(int)
    
    return Y
    
def train_model(Y_in, k, m=None, n=None, reg=0, eta=0.01,
                eps=0.0001, max_epochs=300):
    '''
    Takes as input Y_in, a matrix where each row (i, j, Y_ij) gives the
    rating Y_ij of user i for movie j; k, the number of latent factors;
    m, the number of users (inferred from Y if set to None); n, the
    number of movies (inferred from Y if set to None); reg, the strength
    of regularization (set to 0 by default); eta, the learning rate
    (set to 0.01 by default); eps, the minimum improvement in mean
    regularized squared error at each epoch as a proportion of the
    improvement after the first epoch (set to 0.0001 by default); and
    max_epochs, the maximum number of epochs (set to 300 by default).
    
    Returns trained latent factor model given by matrices U and V and
    bias vectors a and b, along with the unregularized mean squared error
    of the model.
    '''
    Y = process_Y(Y_in)
    
    # Extract numbers of users and movies if needed
    if m is None:
        m = len(np.unique(Y[:, 0]))
    if n is None:
        n = len(np.unique(Y[:, 1]))
    
    # Initialize entries of U, V, a, and b to random numbers in [-0.5, 0.5]
    U = np.random.random((k, m)) - 0.5
    V = np.random.random((k, n)) - 0.5
    a = np.random.random(m) - 0.5
    b = np.random.random(n) - 0.5
    mu = np.mean(Y[:, -1])
    
    # Store loss (regularized mean squared error) for each epoch
    loss = np.empty(max_epochs + 1)
    loss[0] = err(Y, U, V, a, b, mu, reg)
    
    # Perform SGD for specified number of epochs (or until termination)
    for epoch in range(max_epochs):
        # Shuffle data
        np.random.shuffle(Y)
        
        # Update U, V, a, and b for each data point
        for (i, j, Y_ij) in Y:
            grad_U, grad_V, grad_a, grad_b = \
                grad(Y_ij, U[:, i], V[:, j], a[i], b[j], mu=mu, reg=reg)
            U[:, i] -= eta * grad_U
            V[:, j] -= eta * grad_V
            a[i] -= eta * grad_a
            b[j] -= eta * grad_b
        
        # Compute loss and check for stopping condition
        loss[epoch + 1] = err(Y, U, V, a, b, mu, reg)
        if (loss[epoch + 1] - loss[epoch]) <= eps * (loss[1] - loss[0]):
            break
    
    # Compute unregularized mean squared error for final model
    err_model = err(Y, U, V, a, b, mu, reg=0)
    return U, V, a, b, err_model

We can then train our model for the MovieLens dataset.  We start by reading in the data.

In [4]:
# Read in data on ratings
Y_train = pd.read_csv(os.path.join('data', 'train.txt'),
                      sep='\t', header=None,
                      names=['User ID', 'Movie ID', 'Rating'])
Y_test = pd.read_csv(os.path.join('data', 'test.txt'),
                     sep='\t', header=None,
                     names=['User ID', 'Movie ID', 'Rating'])

# Read in information on movies
names = ['Movie ID', 'Movie Title', 'Unknown', 'Action',
         'Adventure', 'Animation', 'Children\'s', 'Comedy',
         'Crime', 'Documentary', 'Drama', 'Fantasy',
         'Film-Noir', 'Horror', 'Musical', 'Mystery',
         'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(os.path.join('data', 'movies.txt'),
                     sep='\t', header=None,
                     encoding='latin_1', names=names)
movies['Movie Title'] = movies['Movie Title'].str.strip()

# Separate movie name and year if desired
split_year = False
if split_year:
    movies.loc[266, 'Movie Title'] = 'unknown (0000)'
    movies.loc[1411, 'Movie Title'] = \
        'Land Before Time III: The Time of the Great Giving (V) (1995)'
    movies['Year'] = [int(title[-5:-1]) for title in movies['Movie Title']]
    movies['Movie Title'] = [title[:-7] for title in movies['Movie Title']]

# Merge ratings data with movie metadata
data_train = Y_train.merge(movies, how='left', on='Movie ID')
data_test = Y_test.merge(movies, how='left', on='Movie ID')

We then perform hyperparameter selection for regularization strength and learning rate using a subset of the training data.

In [45]:
# Specify regularization strengths and learning rates to test
regs = [0, 0.0001, 0.001, 0.01, 0.1, 1]
etas = [0.0001, 0.001, 0.01, 0.1]

# Specify number of latent factors
k = 20

# Extract subset of the training data
Y_sub = Y_train.values[np.random.choice(len(Y_train), 20000, replace=False)]
Y_sub = process_Y(Y_sub)
Y_sub_train = Y_sub[:18000]
Y_sub_test = Y_sub[18000:]

# Save training and testing errors
err_train = np.zeros((len(regs), len(etas)))
err_test = np.zeros((len(regs), len(etas)))

# Train model for each set of hyperparameters
mu = np.mean(Y_sub_train[:, -1])
for i, reg in enumerate(regs):
    for j, eta in enumerate(etas):
        print('Regularization {} and learning rate {}'.format(reg, eta))
        U, V, a, b, err_model = train_model(Y_sub_train, k=k,
                                            m = len(np.unique(Y_sub[:, 0])),
                                            n = len(np.unique(Y_sub[:, 1])),
                                            reg=reg, eta=eta,
                                            eps=0.0001, max_epochs=300)
        err_train[i, j] = err_model
        err_test[i, j] = err(Y_sub_test, U, V, a, b, mu=mu, reg=reg)

Regularization 0 and learning rate 0.0001
Regularization 0 and learning rate 0.001
Regularization 0 and learning rate 0.01
Regularization 0 and learning rate 0.1
Regularization 0.0001 and learning rate 0.0001
Regularization 0.0001 and learning rate 0.001
Regularization 0.0001 and learning rate 0.01
Regularization 0.0001 and learning rate 0.1
Regularization 0.001 and learning rate 0.0001
Regularization 0.001 and learning rate 0.001
Regularization 0.001 and learning rate 0.01
Regularization 0.001 and learning rate 0.1
Regularization 0.01 and learning rate 0.0001
Regularization 0.01 and learning rate 0.001
Regularization 0.01 and learning rate 0.01
Regularization 0.01 and learning rate 0.1
Regularization 0.1 and learning rate 0.0001
Regularization 0.1 and learning rate 0.001
Regularization 0.1 and learning rate 0.01
Regularization 0.1 and learning rate 0.1
Regularization 1 and learning rate 0.0001
Regularization 1 and learning rate 0.001
Regularization 1 and learning rate 0.01
Regularizat

In [ ]:
Y_sub_train

In [47]:
err_train

array([[0.78192849, 0.77260038, 0.60141646, 0.27591038],
       [0.78757331, 0.75623376, 0.59729168, 0.27851863],
       [0.77741859, 0.76951895, 0.59436938, 0.28177306],
       [0.7719383 , 0.75038532, 0.59620854, 0.28945989],
       [0.78444286, 0.75088619, 0.58907561, 0.32420093],
       [0.81259424, 0.74731995, 0.56830247, 0.46332773]])

In [48]:
err_test

array([[0.78449764, 0.78337127, 0.67221776, 0.55908496],
       [0.81155827, 0.78640541, 0.66453664, 0.55821759],
       [0.79041615, 0.80670676, 0.66921852, 0.57985224],
       [0.81319759, 0.7775822 , 0.6727094 , 0.5922801 ],
       [0.83163458, 0.78059501, 0.67244226, 0.54453889],
       [0.82892163, 0.80731727, 0.64943603, 0.55102436]])

In [ ]:
def main():
    print("Factorizing with ", M, " users, ", N, " movies.")
    Ks = [10,20,30,50,100]
	
    reg = 0.0
    eta = 0.03 # learning rate
    E_in = []
    E_out = []
	
    # Use to compute Ein and Eout
    for K in Ks:
        U,V, err = train_model(M, N, K, eta, reg, Y_train)
        E_in.append(err)
        E_out.append(get_err(U, V, Y_test))
	
    plt.plot(Ks, E_in, label='$E_{in}$')
    plt.plot(Ks, E_out, label='$E_{out}$')
    plt.title('Error vs. K')
    plt.xlabel('K')
    plt.ylabel('Error')
    plt.legend()
    plt.savefig('2D.png')

   

if __name__ == "__main__":
    main()
